

---


Below are the list of problems worked using NEAT algorithm

---

1. XOR PROBLEM
2. KNAPSACK PROBLEM
3. TRAVELLING SALESMAN PROBLEM
4. MNIST DATA
5. POLE BALANCING PROBLEM

---


Observations:


---


*   NEAT ALGORITHM FOR NP HARD PROBLEM - TSP IS OBSERVED TO WORK WITH 3000 locations.

*   FOR MNIST DATASET, PARAMETERS NEED FURTHER TUNING

*   ALGORITHM CAN BE FURTHER APPLIED TO DIFFERENT KINDS OF DATA WITH APPROPRIATE FUNCTIONS


# IMPLEMENTING NEAT FOR XOR PROBLEM (FROM THE PAPER)

In [ ]:
import random
import math

# Define the XOR problem inputs and outputs
xor_inputs = [(0, 0), (0, 1), (1, 0), (1, 1)]
xor_outputs = [0, 1, 1, 0]

# Define the activation function (sigmoid)
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Define the genome class
class Genome:
    def __init__(self):
        self.weights = [random.uniform(-1, 1) for _ in range(3)]

    def activate(self, inputs):
        sum = self.weights[0]
        for i in range(2):
            sum += inputs[i] * self.weights[i+1]
        return sigmoid(sum)

# Create the initial population of genomes
population_size = 150
population = [Genome() for _ in range(population_size)]

# Evolution loop
num_generations = 50
for generation in range(num_generations):
    # Evaluate fitness for each genome
    fitness_scores = []
    for genome in population:
        fitness = 4.0
        for inputs, output in zip(xor_inputs, xor_outputs):
            net_output = genome.activate(inputs)
            fitness -= abs(net_output - output)
        fitness_scores.append(fitness)

    # Select parents for reproduction
    parents = random.choices(population, weights=fitness_scores, k=2)

    # Create offspring through crossover and mutation
    offspring = []
    for _ in range(population_size):
        child = Genome()
        for i in range(3):
            parent = random.choice(parents)
            child.weights[i] = parent.weights[i]
            if random.random() < 0.1:  # Mutation
                child.weights[i] += random.uniform(-0.5, 0.5)
        offspring.append(child)

    # Replace old population with offspring
    population = offspring

# Evaluate fitness of the final population
fitness_scores = []
for genome in population:
    fitness = 4.0
    for inputs, output in zip(xor_inputs, xor_outputs):
        net_output = genome.activate(inputs)
        fitness -= abs(net_output - output)
    fitness_scores.append(fitness)

# Find the best genome
best_genome = population[fitness_scores.index(max(fitness_scores))]

# Display the best genome
print("Best genome weights:", best_genome.weights)


Best genome weights: [-0.33312669430355746, -1.4178895341190987, 0.9289147625377027]


# IMPLEMENTING NEAT FOR KNAPSACK PROBLEM

In [ ]:
import random
import numpy as np

# Constants
POPULATION_SIZE = 150
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 5
ELITISM_PERCENTAGE = 0.1
SPECIES_THRESHOLD = 3

class Genome:
    def __init__(self, size):
        self.size = size
        self.fitness = 0.0
        self.adjusted_fitness = 0.0
        self.genes = [random.randint(0, 1) for _ in range(size)]

    def copy(self):
        copied_genome = Genome(self.size)
        copied_genome.fitness = self.fitness
        copied_genome.adjusted_fitness = self.adjusted_fitness
        copied_genome.genes = self.genes.copy()
        return copied_genome

class Species:
    def __init__(self):
        self.genomes = []

class NEAT:
    def __init__(self, capacity, weights, values):
        self.population = []
        self.species = []
        self.current_generation = 1
        self.capacity = capacity
        self.weights = weights
        self.values = values

    def initialize_population(self, size):
        for _ in range(POPULATION_SIZE):
            genome = Genome(size)
            self.population.append(genome)

    def run(self):
        size = len(self.weights)
        self.initialize_population(size)

        for generation in range(1, MAX_GENERATIONS + 1):
            print(f"Generation {generation} - Best Fitness: {self.get_best_genome().fitness}")
            self.evaluate_fitness()
            self.adjust_fitness()
            self.reproduce()
            self.current_generation += 1

        return self.get_best_genome()

    def evaluate_fitness(self):
        # Evaluate fitness for each genome
        for genome in self.population:
            total_value = sum(self.values[i] for i in range(len(genome.genes)) if genome.genes[i] == 1)
            total_weight = sum(self.weights[i] for i in range(len(genome.genes)) if genome.genes[i] == 1)

            if total_weight > self.capacity:
                genome.fitness = 0.0
            else:
                genome.fitness = total_value

    def adjust_fitness(self):
        # Adjust fitness based on species
        for species in self.species:
            total_fitness = sum(genome.fitness for genome in species.genomes)
            average_fitness = total_fitness / len(species.genomes)
            for genome in species.genomes:
                genome.adjusted_fitness = genome.fitness / average_fitness

    def reproduce(self):
        new_population = []

        # Preserve a certain percentage of top-performing genomes (Elitism)
        elite_count = int(ELITISM_PERCENTAGE * POPULATION_SIZE)
        elites = sorted(self.population, key=lambda g: g.fitness, reverse=True)[:elite_count]
        new_population.extend(elites)

        # Create new offspring through reproduction
        while len(new_population) < POPULATION_SIZE:
            parent1 = self.select_parent()
            parent2 = self.select_parent()
            offspring = self.crossover(parent1, parent2)
            self.mutate(offspring)
            new_population.append(offspring)

        # Replace the old population with the new population
        self.population = new_population

    def select_parent(self):
        tournament_pool = random.sample(self.population, TOURNAMENT_SIZE)
        return max(tournament_pool, key=lambda g: g.fitness)

    def crossover(self, parent1, parent2):
        if parent2.fitness > parent1.fitness:
            parent1, parent2 = parent2, parent1

        offspring = parent1.copy()

        for i in range(offspring.size):
            if random.random() < 0.5:
                offspring.genes[i] = parent2.genes[i]

        return offspring

    def mutate(self, genome):
        for i in range(genome.size):
            if random.random() < MUTATION_RATE:
                genome.genes[i] = 1 - genome.genes[i]

    def get_best_genome(self):
        return max(self.population, key=lambda g: g.fitness)


# Example usage
weights = [3, 2, 5, 4, 1]
values = [5, 4, 8, 6, 3]
capacity = 10

knapsack_neat = NEAT(capacity, weights, values)
best_genome = knapsack_neat.run()

# Print the best genome
print("Best Genome:", best_genome.genes)
print("Best Genome Fitness:", best_genome.fitness)


Generation 1 - Best Fitness: 0.0
Generation 2 - Best Fitness: 18
Generation 3 - Best Fitness: 18
Generation 4 - Best Fitness: 18
Generation 5 - Best Fitness: 18
Generation 6 - Best Fitness: 18
Generation 7 - Best Fitness: 18
Generation 8 - Best Fitness: 18
Generation 9 - Best Fitness: 18
Generation 10 - Best Fitness: 18
Generation 11 - Best Fitness: 18
Generation 12 - Best Fitness: 18
Generation 13 - Best Fitness: 18
Generation 14 - Best Fitness: 18
Generation 15 - Best Fitness: 18
Generation 16 - Best Fitness: 18
Generation 17 - Best Fitness: 18
Generation 18 - Best Fitness: 18
Generation 19 - Best Fitness: 18
Generation 20 - Best Fitness: 18
Generation 21 - Best Fitness: 18
Generation 22 - Best Fitness: 18
Generation 23 - Best Fitness: 18
Generation 24 - Best Fitness: 18
Generation 25 - Best Fitness: 18
Generation 26 - Best Fitness: 18
Generation 27 - Best Fitness: 18
Generation 28 - Best Fitness: 18
Generation 29 - Best Fitness: 18
Generation 30 - Best Fitness: 18
Generation 31 - Be

# IMPLEMENTING NEAT FOR TRAVELLING SALESMAN PROBLEM 4 LOCATIONS

In [ ]:
import random
import numpy as np
import math

# Constants
POPULATION_SIZE = 150
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 5
ELITISM_PERCENTAGE = 0.1
SPECIES_THRESHOLD = 3

class Genome:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.fitness = 0.0
        self.adjusted_fitness = 0.0
        self.path = list(range(num_cities))
        random.shuffle(self.path)

    def copy(self):
        copied_genome = Genome(self.num_cities)
        copied_genome.fitness = self.fitness
        copied_genome.adjusted_fitness = self.adjusted_fitness
        copied_genome.path = self.path.copy()
        return copied_genome

class Species:
    def __init__(self):
        self.genomes = []

class NEAT:
    def __init__(self, distance_matrix):
        self.population = []
        self.species = []
        self.current_generation = 1
        self.num_cities = len(distance_matrix)
        self.distance_matrix = distance_matrix

    def initialize_population(self, num_cities):
        for _ in range(POPULATION_SIZE):
            genome = Genome(num_cities)
            self.population.append(genome)

    def run(self):
        num_cities = len(self.distance_matrix)
        self.initialize_population(num_cities)

        for generation in range(1, MAX_GENERATIONS + 1):
            print(f"Generation {generation} - Best Fitness: {self.get_best_genome().fitness}")
            self.evaluate_fitness()
            self.adjust_fitness()
            self.reproduce()
            self.current_generation += 1

        return self.get_best_genome()

    def evaluate_fitness(self):
        # Evaluate fitness for each genome
        for genome in self.population:
            total_distance = 0
            for i in range(self.num_cities - 1):
                city1 = genome.path[i]
                city2 = genome.path[i + 1]
                total_distance += self.distance_matrix[city1][city2]
            total_distance += self.distance_matrix[genome.path[-1]][genome.path[0]]
            genome.fitness = 1 / total_distance

    def adjust_fitness(self):
        # Adjust fitness based on species
        for species in self.species:
            total_fitness = sum(genome.fitness for genome in species.genomes)
            average_fitness = total_fitness / len(species.genomes)
            for genome in species.genomes:
                genome.adjusted_fitness = genome.fitness / average_fitness

    def reproduce(self):
        new_population = []

        # Preserve a certain percentage of top-performing genomes (Elitism)
        elite_count = int(ELITISM_PERCENTAGE * POPULATION_SIZE)
        elites = sorted(self.population, key=lambda g: g.fitness, reverse=True)[:elite_count]
        new_population.extend(elites)

        # Create new offspring through reproduction
        while len(new_population) < POPULATION_SIZE:
            parent1 = self.select_parent()
            parent2 = self.select_parent()
            offspring = self.crossover(parent1, parent2)
            self.mutate(offspring)
            new_population.append(offspring)

        # Replace the old population with the new population
        self.population = new_population

    def select_parent(self):
        tournament_pool = random.sample(self.population, TOURNAMENT_SIZE)
        return max(tournament_pool, key=lambda g: g.fitness)

    def crossover(self, parent1, parent2):
        if parent2.fitness > parent1.fitness:
            parent1, parent2 = parent2, parent1

        offspring = parent1.copy()

        # Choose a random subset of cities from parent2
        subset_size = random.randint(2, self.num_cities - 1)
        subset_indices = random.sample(range(self.num_cities), subset_size)
        subset = [parent2.path[i] for i in subset_indices]

        # Insert the subset into offspring while maintaining the order of other cities
        for city in subset:
            index = offspring.path.index(city)
            offspring.path.remove(city)
            offspring.path.insert(index, city)

        return offspring

    def mutate(self, genome):
        for i in range(self.num_cities):
            if random.random() < MUTATION_RATE:
                j = random.randint(0, self.num_cities - 1)
                genome.path[i], genome.path[j] = genome.path[j], genome.path[i]

    def get_best_genome(self):
        return max(self.population, key=lambda g: g.fitness)


# Example usage
distance_matrix = [
    [0, 2, 9, 10],
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
]

tsp_neat = NEAT(distance_matrix)
best_genome = tsp_neat.run()

# Print the best genome
print("Best Genome:", best_genome.path)
print("Best Genome Fitness:", best_genome.fitness)


Generation 1 - Best Fitness: 0.0
Generation 2 - Best Fitness: 0.047619047619047616
Generation 3 - Best Fitness: 0.047619047619047616
Generation 4 - Best Fitness: 0.047619047619047616
Generation 5 - Best Fitness: 0.047619047619047616
Generation 6 - Best Fitness: 0.047619047619047616
Generation 7 - Best Fitness: 0.047619047619047616
Generation 8 - Best Fitness: 0.047619047619047616
Generation 9 - Best Fitness: 0.047619047619047616
Generation 10 - Best Fitness: 0.047619047619047616
Generation 11 - Best Fitness: 0.047619047619047616
Generation 12 - Best Fitness: 0.047619047619047616
Generation 13 - Best Fitness: 0.047619047619047616
Generation 14 - Best Fitness: 0.047619047619047616
Generation 15 - Best Fitness: 0.047619047619047616
Generation 16 - Best Fitness: 0.047619047619047616
Generation 17 - Best Fitness: 0.047619047619047616
Generation 18 - Best Fitness: 0.047619047619047616
Generation 19 - Best Fitness: 0.047619047619047616
Generation 20 - Best Fitness: 0.047619047619047616
Genera

# IMPLEMENTING NEAT FOR TSP FOR 3000 LOCATIONS

In [ ]:
import random
import numpy as np
import math

# Constants
POPULATION_SIZE = 150
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 5
ELITISM_PERCENTAGE = 0.1
SPECIES_THRESHOLD = 3

class Genome:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.fitness = 0.0
        self.adjusted_fitness = 0.0
        self.path = list(range(num_cities))
        random.shuffle(self.path)

    def copy(self):
        copied_genome = Genome(self.num_cities)
        copied_genome.fitness = self.fitness
        copied_genome.adjusted_fitness = self.adjusted_fitness
        copied_genome.path = self.path.copy()
        return copied_genome

class Species:
    def __init__(self):
        self.genomes = []

class NEAT:
    def __init__(self, distance_matrix):
        self.population = []
        self.species = []
        self.current_generation = 1
        self.num_cities = len(distance_matrix)
        self.distance_matrix = distance_matrix

    def initialize_population(self, num_cities):
        for _ in range(POPULATION_SIZE):
            genome = Genome(num_cities)
            self.population.append(genome)

    def run(self):
        num_cities = len(self.distance_matrix)
        self.initialize_population(num_cities)

        for generation in range(1, MAX_GENERATIONS + 1):
            print(f"Generation {generation} - Best Fitness: {self.get_best_genome().fitness}")
            self.evaluate_fitness()
            self.adjust_fitness()
            self.reproduce()
            self.current_generation += 1

        return self.get_best_genome()

    def evaluate_fitness(self):
        # Evaluate fitness for each genome
        for genome in self.population:
            total_distance = 0
            for i in range(self.num_cities - 1):
                city1 = genome.path[i]
                city2 = genome.path[i + 1]
                total_distance += self.distance_matrix[city1][city2]
            total_distance += self.distance_matrix[genome.path[-1]][genome.path[0]]
            genome.fitness = 1 / total_distance

    def adjust_fitness(self):
        # Adjust fitness based on species
        for species in self.species:
            total_fitness = sum(genome.fitness for genome in species.genomes)
            average_fitness = total_fitness / len(species.genomes)
            for genome in species.genomes:
                genome.adjusted_fitness = genome.fitness / average_fitness

    def reproduce(self):
        new_population = []

        # Preserve a certain percentage of top-performing genomes (Elitism)
        elite_count = int(ELITISM_PERCENTAGE * POPULATION_SIZE)
        elites = sorted(self.population, key=lambda g: g.fitness, reverse=True)[:elite_count]
        new_population.extend(elites)

        # Create new offspring through reproduction
        while len(new_population) < POPULATION_SIZE:
            parent1 = self.select_parent()
            parent2 = self.select_parent()
            offspring = self.crossover(parent1, parent2)
            self.mutate(offspring)
            new_population.append(offspring)

        # Replace the old population with the new population
        self.population = new_population

    def select_parent(self):
        tournament_pool = random.sample(self.population, TOURNAMENT_SIZE)
        return max(tournament_pool, key=lambda g: g.fitness)

    def crossover(self, parent1, parent2):
        if parent2.fitness > parent1.fitness:
            parent1, parent2 = parent2, parent1

        offspring = parent1.copy()

        # Choose a random subset of cities from parent2
        subset_size = random.randint(2, self.num_cities - 1)
        subset_indices = random.sample(range(self.num_cities), subset_size)
        subset = [parent2.path[i] for i in subset_indices]

        # Insert the subset into offspring while maintaining the order of other cities
        for city in subset:
            index = offspring.path.index(city)
            offspring.path.remove(city)
            offspring.path.insert(index, city)

        return offspring

    def mutate(self, genome):
        for i in range(self.num_cities):
            if random.random() < MUTATION_RATE:
                j = random.randint(0, self.num_cities - 1)
                genome.path[i], genome.path[j] = genome.path[j], genome.path[i]

    def get_best_genome(self):
        return max(self.population, key=lambda g: g.fitness)

# Generate a random distance matrix for 1000 cities
num_cities = 3000
distance_matrix = np.zeros((num_cities, num_cities))
for i in range(num_cities):
    for j in range(i + 1, num_cities):
        distance = random.randint(1, 100)
        distance_matrix[i][j] = distance
        distance_matrix[j][i] = distance

tsp_neat = NEAT(distance_matrix)
best_genome = tsp_neat.run()

# Print the best genome
print("Best Genome:", best_genome.path)
print("Best Genome Fitness:", best_genome.fitness)

Generation 1 - Best Fitness: 0.0
Generation 2 - Best Fitness: 6.763474532136649e-06
Generation 3 - Best Fitness: 6.833308277869648e-06
Generation 4 - Best Fitness: 6.860027988914195e-06
Generation 5 - Best Fitness: 6.882644036533074e-06
Generation 6 - Best Fitness: 6.882644036533074e-06
Generation 7 - Best Fitness: 6.922091856158932e-06
Generation 8 - Best Fitness: 6.940973957465712e-06
Generation 9 - Best Fitness: 6.940973957465712e-06
Generation 10 - Best Fitness: 6.940973957465712e-06
Generation 11 - Best Fitness: 6.988657409025152e-06
Generation 12 - Best Fitness: 6.988657409025152e-06
Generation 13 - Best Fitness: 6.988657409025152e-06
Generation 14 - Best Fitness: 6.995012556047538e-06
Generation 15 - Best Fitness: 6.995012556047538e-06
Generation 16 - Best Fitness: 7.033288554729535e-06
Generation 17 - Best Fitness: 7.033288554729535e-06
Generation 18 - Best Fitness: 7.036455877903418e-06
Generation 19 - Best Fitness: 7.036455877903418e-06
Generation 20 - Best Fitness: 7.0364558

# IMPLEMENTING NEAT FOR MNIST DATASET

In [ ]:
import tensorflow as tf

In [ ]:
if tf.test.is_gpu_available():
    print("GPU is available")
else:
    print("GPU is not available")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU is available


In [ ]:
if tf.test.is_gpu_available():
    # Set the desired GPU device (e.g., GPU device 0)
    tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda_11.0.3_450.51.06_linux.run
!chmod +x cuda_11.0.3_450.51.06_linux.run
!sudo ./cuda_11.0.3_450.51.06_linux.run

--2023-06-05 05:22:14--  https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda_11.0.3_450.51.06_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3112522594 (2.9G) [application/octet-stream]
Saving to: ‘cuda_11.0.3_450.51.06_linux.run’

cuda_11.0.3_450.51. 100%[===================>]   2.90G   239MB/s    in 16s     

2023-06-05 05:22:30 (187 MB/s) - ‘cuda_11.0.3_450.51.06_linux.run’ saved [3112522594/3112522594]

)07=
   End User License Agreement
   --------------------------

   NVIDIA Software License Agreement and CUDA Supplement to
   Software License Agreement.


   Preface
   -------

   The Software License Agreement in Chapter 1 and the Supplement
   in Chapter 2 contain license terms and conditions that govern
   the use of NVIDIA softwa

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import random
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, initializers

# Constants
INPUT_SIZE = 784
OUTPUT_SIZE = 10
POPULATION_SIZE = 150
MAX_GENERATIONS = 10
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 5
ELITISM_PERCENTAGE = 0.1
SPECIES_THRESHOLD = 3

class ConnectionGene:
    def __init__(self, in_node, out_node, weight):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.enabled = True
        self.innovation_number = None

    def copy(self):
        return ConnectionGene(self.in_node, self.out_node, self.weight)

class Genome:
    def __init__(self):
        self.connections = {}
        self.accuracy = 0.0
        self.adjusted_accuracy = 0.0

    def copy(self):
        copied_genome = Genome()
        copied_genome.connections = {connection_id: connection.copy() for connection_id, connection in self.connections.items()}
        copied_genome.accuracy = self.accuracy
        copied_genome.adjusted_accuracy = self.adjusted_accuracy
        return copied_genome

class Species:
    def __init__(self):
        self.genomes = []

class NEAT:
    def __init__(self):
        self.population = []
        self.species = []
        self.current_generation = 1
        self.innovation_number = 0

    def initialize_population(self):
        for _ in range(POPULATION_SIZE):
            genome = Genome()
            self.create_connections(genome)
            self.population.append(genome)

    def create_connections(self, genome):
        for i in range(INPUT_SIZE):
            for j in range(INPUT_SIZE, INPUT_SIZE + OUTPUT_SIZE):
                connection = ConnectionGene(i, j, random.uniform(-1, 1))
                connection.innovation_number = self.innovation_number
                genome.connections[self.innovation_number] = connection
                self.innovation_number += 1

    def run(self):
        self.initialize_population()

        for generation in range(1, MAX_GENERATIONS + 1):
            print(f"Generation {generation} - Best Accuracy: {self.get_best_genome().accuracy}")
            self.evaluate_accuracy()
            self.adjust_accuracy()
            self.reproduce()
            self.current_generation += 1

        return self.get_best_genome()

    def evaluate_accuracy(self):
        # Load MNIST dataset
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        for genome in self.population:
            # Initialize neural network with the genome's connections
            network = initialize_neural_network(genome)

            # Flatten and normalize the input images
            x_train_flat = x_train.reshape(-1, INPUT_SIZE) / 255.0
            x_test_flat = x_test.reshape(-1, INPUT_SIZE) / 255.0

            # Perform forward propagation and calculate accuracy
            predictions = network.predict(x_test_flat)
            predicted_labels = np.argmax(predictions, axis=1)
            accuracy = np.mean(predicted_labels == y_test)

            # Assign accuracy score to the genome
            genome.accuracy = accuracy

    def adjust_accuracy(self):
        # Adjust accuracy based on species
        for species in self.species:
            total_accuracy = sum(genome.accuracy for genome in species.genomes)
            average_accuracy = total_accuracy / len(species.genomes)

            for genome in species.genomes:
                genome.adjusted_accuracy = genome.accuracy / average_accuracy

    def reproduce(self):
        new_population = []

        # Preserve a certain percentage of top-performing genomes (Elitism)
        elite_count = int(ELITISM_PERCENTAGE * POPULATION_SIZE)
        elites = sorted(self.population, key=lambda g: g.accuracy, reverse=True)[:elite_count]
        new_population.extend(elites)

        # Create new offspring through reproduction
        while len(new_population) < POPULATION_SIZE:
            parent1 = self.select_parent()
            parent2 = self.select_parent()
            offspring = self.crossover(parent1, parent2)
            self.mutate(offspring)
            new_population.append(offspring)

        # Replace the old population with the new population
        self.population = new_population

    def select_parent(self):
        tournament_pool = random.sample(self.population, TOURNAMENT_SIZE)
        return max(tournament_pool, key=lambda g: g.adjusted_accuracy)

    def crossover(self, parent1, parent2):
        if parent2.accuracy > parent1.accuracy:
            parent1, parent2 = parent2, parent1

        offspring = parent1.copy()

        for connection_id, connection in parent2.connections.items():
            if connection_id not in offspring.connections:
                offspring.connections[connection_id] = connection.copy()
            elif random.choice([True, False]):
                offspring.connections[connection_id] = connection.copy()

        return offspring

    def mutate(self, genome):
        for connection in genome.connections.values():
            if random.random() < MUTATION_RATE:
                connection.weight += random.uniform(-1, 1)

    def get_best_genome(self):
        return max(self.population, key=lambda g: g.accuracy)

def initialize_neural_network(genome):
    # Create input layer
    input_layer = layers.Input(shape=(INPUT_SIZE,))

    # Create a dictionary to hold the layers
    layers_dict = {}

    # Iterate through genome connections and create corresponding layers
    for connection_id, connection in genome.connections.items():
        if connection.enabled:
            if connection.in_node not in layers_dict:
                # Create a new layer if the in_node doesn't exist in layers_dict
                layers_dict[connection.in_node] = layers.Dense(
                    1,
                    kernel_initializer=initializers.Constant(connection.weight),
                    bias_initializer='zeros',
                    activation='sigmoid'
                )(input_layer)
            # Create a dense layer for each enabled connection
            dense_layer = layers.Dense(
                1,
                kernel_initializer=initializers.Constant(connection.weight),
                bias_initializer='zeros',
                activation='sigmoid'
            )(layers_dict[connection.in_node])

            layers_dict[connection.out_node] = dense_layer

    # Concatenate the output layers
    if layers_dict:
        output_layers = [layer for node_id, layer in layers_dict.items() if node_id >= INPUT_SIZE]
        concatenated_output = layers.concatenate(output_layers)
    else:
        concatenated_output = input_layer

    # Create the output layer
    output_layer = layers.Dense(OUTPUT_SIZE, activation='softmax')(concatenated_output)

    # Create the model using the input and output layers
    model = models.Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Example usage
mnist_neat = NEAT()
best_genome = mnist_neat.run()

# Evaluate the best genome on the test data
print("Best Genome Accuracy:", best_genome.accuracy)


# IMPLEMENTING NEAT FOR POLE BALANCING PROBLEM


In [ ]:
import numpy as np
import random
import gym


class NeuralNetwork:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.random.randn(output_size, 1)

    def forward(self, inputs):
        inputs = np.array(inputs).reshape(-1, 1)
        outputs = np.dot(self.weights, inputs) + self.biases
        return outputs

    def mutate(self, mutation_rate):
        def apply_mutation(x):
            if random.random() < mutation_rate:
                return x + np.random.randn()
            return x

        self.weights = np.vectorize(apply_mutation)(self.weights)
        self.biases = np.vectorize(apply_mutation)(self.biases)


class Genome:
    def __init__(self, input_size, output_size):
        self.neural_network = NeuralNetwork(input_size, output_size)
        self.fitness = 0.0


def crossover(parent1, parent2):
    child = Genome(parent1.neural_network.input_size, parent1.neural_network.output_size)

    for i in range(child.neural_network.output_size):
        if random.random() < 0.5:
            child.neural_network.weights[i] = parent1.neural_network.weights[i]
            child.neural_network.biases[i] = parent1.neural_network.biases[i]
        else:
            child.neural_network.weights[i] = parent2.neural_network.weights[i]
            child.neural_network.biases[i] = parent2.neural_network.biases[i]

    return child


def mutate(genome, mutation_rate):
    genome.neural_network.mutate(mutation_rate)


def evaluate_genome(genome, max_steps=40000):
    env = gym.make("CartPole-v1")
    observation = env.reset()

    for _ in range(max_steps):
        action = np.argmax(genome.neural_network.forward(observation))
        observation, reward, done, _ = env.step(action)
        genome.fitness += reward

        if done:
            break

    env.close()


def create_initial_population(population_size, input_size, output_size):
    population = []

    for _ in range(population_size):
        genome = Genome(input_size, output_size)
        population.append(genome)

    return population


def evolve(population, elite_percentage, mutation_rate):
    population.sort(key=lambda x: x.fitness, reverse=True)
    elite_count = int(elite_percentage * len(population))
    elites = population[:elite_count]
    offspring = elites.copy()

    while len(offspring) < len(population):
        parent1 = random.choice(elites)
        parent2 = random.choice(elites)
        child = crossover(parent1, parent2)
        mutate(child, mutation_rate)
        offspring.append(child)

    return offspring


def run_neat(population_size, input_size, output_size, generations, elite_percentage, mutation_rate):
    population = create_initial_population(population_size, input_size, output_size)

    for generation in range(generations):
        print(f"Generation {generation + 1}")

        for genome in population:
            genome.fitness = 0.0
            evaluate_genome(genome)

        best_genome = max(population, key=lambda x: x.fitness)
        print(f"Best fitness: {best_genome.fitness}")

        population = evolve(population, elite_percentage, mutation_rate)

    best_genome = max(population, key=lambda x: x.fitness)
    print(f"\nBest genome fitness: {best_genome.fitness}")


if __name__ == "__main__":
    run_neat(population_size=50, input_size=4, output_size=2, generations=5, elite_percentage=0.2, mutation_rate=0.1)


Generation 1
Best fitness: 37.0
Generation 2
Best fitness: 284.0
Generation 3
Best fitness: 126.0
Generation 4
Best fitness: 499.0
Generation 5
Best fitness: 500.0

Best genome fitness: 500.0
